# Multi-modal generation pipeline
## Image, Captions, and a Video
In this notebook you will learn how to using Stable Diffusion via LangChain to generate an image, a caption for your choice of `llm`, and finally generate a video based on a generated "*improved prompt*".

## Get Started
**Installation**:

In [ ]:
%pip install -Uq gradio_tools langchain pillow openai

**Enable LangSmith tracing**:

In [ ]:
import os
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "generate-and-caption"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from gradio_tools.tools import (
    ImageCaptioningTool,
    StableDiffusionPromptGeneratorTool,
    StableDiffusionTool,
    TextToVideoTool,
)
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from PIL import Image
from IPython.display import display

In [ ]:
# Set up paths
IMAGE_PATH = 'generated_image.jpg'
CAPTION_PATH = 'generated_caption.txt'

### StableDiffusionTool()
First, let's test the image generation capabilities.

In [ ]:
local_file_path = StableDiffusionTool().langchain.run(
    "Please create a selfie of Spiderman 2099, miguel ohara, skydiving stunt in New York City Skyline"
)
im = Image.open(local_file_path)
display(im)

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history")
tools = [
    StableDiffusionTool().langchain,
    ImageCaptioningTool().langchain,
    StableDiffusionPromptGeneratorTool().langchain,
    TextToVideoTool().langchain,
]


agent = initialize_agent(
    tools, llm, memory=memory, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)
output = agent.run(
    input=(
        "Please create a black chiweenie riding a skateboard"
        "but improve my prompt prior to using an image generator."
        "Use the exact prompt you wrote for the image generator to generate a video."
    )
)